# pyradon package demo

Start by importing the simulator module and making a simulator object

In [ ]:
from src.simulator import Simulator

s = Simulator()
s.debug_bit = 2


Now let us give the simulator some start and end points. 

In [ ]:
s.x1 = 0.375; s.x2 = 0.5; s.y1 = 0; s.y2 = 0.5;
s.run()

Looks like it finds the correct coordinates. Lets show it. 

In [ ]:
import matplotlib.pyplot as plt

%matplotlib widget
streak = s.finder.streaks[0]
offset = 15
plt.figure()
plt.imshow(s.image)
plt.plot([streak.x1-offset, streak.x2-offset], [streak.y1,streak.y2], lw=1, ls='--', color='red')
plt.plot([streak.x1+offset, streak.x2+offset], [streak.y1,streak.y2], lw=1, ls='--', color='red')
plt.colorbar()
plt.show()

What if we have an external image? First we generate it from the Simulator (but it can be a real image)

In [ ]:
s.x1 = 0.2; s.x2 = 0.6; s.y1 = 0; s.y2 = 0.5; s.run()

Lets create some images from the noiseless "image_conv" in the Simulator.

In [ ]:
plt.figure()
plt.imshow(s.image_clean)
plt.colorbar();

In [ ]:
import numpy as np
I = np.random.normal(103, 2.5, (10,512,512))
I[2,:,:] = I[2,:,:] + s.image_clean

In [ ]:
plt.figure()
plt.imshow(I[2,:,:])
plt.colorbar();

We now have 10 noisy images with the line in image number 3.
Now lets put these images into the streak finder:

In [ ]:
s.finder.pars.use_exclude = False
s.finder.pars.use_subtract_mean = True
s.finder.pars.use_short = True
s.finder.reset()
for i in range(I.shape[0]): 
    s.finder.input(I[i,:,:], psf=2, variance=2.5**2) # note the psf is given as width sigma, and variance of the noise we used. 
    print(f'Frame= {i} | Best S/N= {s.finder.data.best_snr}')

if len(s.finder.streaks_all):
    s.finder.streaks_all[0].print()

Note how the S/N of all the frames is below 5, while the streak is detected at S/N=26. Compare that with the expected S/N: 

In [ ]:
s.bg_noise_var = 2.5**2
s.calc_snr()

Additional inputs to the Finder.input() method include:  
-filename, batch_num: To keep track of where each streak was found   
-psf: as a scalar (PSF width $\sigma$) or a 2D map   
-variance: a scalar (average background variance) or a 2D map   

Some of the parameters of Finder that can be used to improve streak detection:   
-use_subtract_mean=1: each image mean is subtracted, ignoring NaNs (NaNs are then replaced with zeros).
-use_conv=1: choose if to convolve the image before doing FRT.   
-use_sections=0: use this to cut the image into smaller sections to run faster. Might fail to find faint and long streaks. 
-size_sections=1024: the size of the square sections. 
-offset_sections=(0,0): a starting point for making sections (usefull to trim edges). 
-pad_sections=0: wether to pad the sections if any of them reach outside the image area. 
-use_short=1: find short streaks.  
-min_length=32: shortest streaks to save (pixels along the horizontal/vertical)  
-threshold=5: in units of S/N.  
-num_iterations=5: apply FRT again and again, subtracting the streaks each time.   
-subtract_psf_widths=3: how many PSF $\sigma$'s to subtract in each direction of the streak.   
-use_exclude=1: Zeros the central part of the Radon image before finding maxima (gets rid of line-noise).   
-exclude_x_pix=[-50,50]: horizontal pixel range (in final Radon image; for partial frames uses a relative size).   
-exclude_y_pix=[]: same as above, for the vertical pixel range.  
-use_write_cutouts=1: saves the cutouts of each found streak. Use 0 to save space.
-max_length_to_write=128: only save streaks shorter than this length. 
-use_show=0: display the images. 
-default_psf_sigma=1: the PSF's gaussian sigma parameter, if no PSF is given. 
-default_var_scalar=1: the noise variance value if no noise is given. 
-verbosity=1: how much debugging data to print while running. 

